### Infêrência Estatística
# Título: Avaliação do impacto da expressões multipalavras nos modelos de linguagem

In [1]:
import markovify
import os
import time
import json
import nltk
import math
import numpy as np

Baseamos o código no Read me da biblioteca "Markovify" em: https://github.com/jsvine/markovify/blob/master/README.md

Para cada arquivos do diretórios especificado lemos e preprocessamos o texto, criamos uma rede de markov com os n-grams (state_size) e combinamos os modelos de todos os arquivos.

In [2]:
# Primeiro, criamos uma função para preprocessar os textos
def preprocess(text):
    # Decodificar usando ISO 8859-1 
    text = text.decode('ISO 8859-1')
    # Colocar todas as letras em minúscula
    text = text.lower()
    # Separar as sentenças pelos pontos de finais
    text = text.split('.')
    new_text = []
    for sent in text:
        # Eliminamos as sentenças com menos de 100 caracteres
        if len(sent)> 100:
            # Excluir os espaços no início e fim das sentenças
            new_text.append(sent.strip())
    return(new_text)

In [3]:
# Contador de tempo
start_time = time.time()

# Cadeia de Markov combinada
combined_model = None

# Contagem de arquivos
n_files = 0

# Iterar pelos diretórios e arquivos
for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        n_files += 1
        
        try:
            # Ler o arquivo e criar uma cadeia de Markov para cada arquivo
            with open(os.path.join(dirpath, filename), 'rb') as f:
                # Preprocessar o texto lido
                text = preprocess(f.read())
                model = markovify.NewlineText(text, retain_original=False, state_size=2)

                # Combinar modelo criado para um arquivo com a Cadeia de Markov combinada
                if combined_model:
                    combined_model = markovify.combine(models=[combined_model, model])
                else:
                    combined_model = model
        except:
            print("Não foi possível ler o arquivo", filename)
            #pass
                    
        # Contador de arquivos processados
        if n_files % 50 == 0:
            print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))

# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50 arquivos processados --- --- 17.93569540977478 seconds ---
--- 100 arquivos processados --- --- 68.74974584579468 seconds ---
--- 124.99825191497803 seconds ---


In [4]:
# Imprimir um exemplo de sentença aleatória
print(combined_model.make_sentence(max_words=50))

   a   p a r t i r   ó l e o   d e   s o j a   a   2 5   ° c ,   4 0   ° c   d e   m i s t u r a s   ó l e o   e   l e v a d o   p a r a   l a b o r a t ó r i o s   d i s t a n t e s 


In [5]:
# Gravar modelo em JSON em disco
model_json = combined_model.to_json()
with open('MarkovChainModel_base.json', 'w') as f:
    json.dump(model_json, f)

In [6]:
# Carregar modelo em JSON do disco
with open('MarkovChainModel_base.json', 'r') as f:
    model_json = markovify.Text.from_json(json.load(f))

Como os textos usados são em português, foi necessário codificar em 'ISO 8859-1', já o modelo JSON está codificado em UTF16. Portanto, ao ler e consultar os termos do modelo é necessário usar a codificação correta.

In [8]:
# Geração de sentença aleatória que iniciam com n-palavras

# Palavras que serãao usadas como semente
palavra_1 = 'a'
palavra_2 = 'petrobras'

# Ajustando a codificação para ser usado no modelo
palavra_1 = ' ' + palavra_1 + ' '
palavra_2 = ' ' + palavra_2
palavra_1 = palavra_1.encode('utf16')[3:]
palavra_2 = palavra_2.encode('utf16')[4:]
seed = palavra_1 + palavra_2

#Criando sentença aleatória iniciando com as palavras sementes
print(combined_model.make_sentence_with_start(seed.decode()))

 a   p e t r o b r a s   a d o t a m   a l i a n ç a s   e s t r a t é g i c a s 


Do modelo criado pela biblioteca Markovify, extraímos a cadeia de Markov e gravamos em um dicionário

In [9]:
# Extrair cadeia de Markov do modelo
chain = combined_model.to_dict()['chain']
# Transformando a cadeia em um dicionário 
dic = {}
for state in json.loads(chain):
    dic[(state[0][0],state[0][1])] = state[1]

Criamos uma função para consultar a cadeia de Markov. Dado duas palavras, extraímos as próximas palavras possíveis, a contagem delas no conjunto de treinamento e a soma total. Dessa forma conseguimos calcular  $p(x_3|x_2,x_1)$.

In [10]:
# Função que dado duas palavras, retorna um dicionários com as probabilidades das palavras seguintes
def query_chain(palavra_1, palavra_2):
    # Selecionar o dicionário cuja entrada é o par de palavras "palavra_1" e "palavra_2"
    d = dic[(palavra_1, palavra_2)]
    new_d = {}
    # Transformar a contagem de palavras em probabilidade
    for w in d:
        new_d[w] = d[w]/sum(d.values())
    return (new_d)

O calculamos a perplexidade para cada sentença. Dada as primeiras palavras, obtivemos do modelo a probabilidade da palavra seguinte. Em seguida, avançamos pela sentença recuperando as probablidades.

In [11]:
# Dada uma sentença, a função retorna uma lista com as entropias calculada para cada trio de palavras
def entropy(sent):
    # Quebra a sentença em tokens
    words = nltk.word_tokenize(sent)
    # Variável para guardar a entropia de cada conjunto de palavras
    ln_prob_Xi = []
    # Janela de iteração pela sentença
    for w in range(len(words)-2):
        # Cálculo a entropia
        try:
            ln_prob_Xi.append(- math.log(query_chain(words[w], words[w+1])[words[w+2]]))
        except:
            pass

    return(np.mean(ln_prob_Xi))

In [12]:
# Função que recebe uma string com o caminho e nome da pasta com os arquivos e retorna a perplexidade

def perplexity(dirpath):

    # Início do contador de tempo
    start_time = time.time()
    
    # Variável para armazenar as entropias
    ent = []

    # Contagem de arquivos
    n_files = 0
    # Iterando pelos arquivos da pasta indicada em
    for (dirpath, _, filenames) in os.walk(dirpath):
        
        for filename in filenames:
            n_files += 1
            try:
                # Ler o arquivo e calcular a entropia
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    # Preprocessar o texto lido
                    text = preprocess(f.read())
                    #Calcular a entropia da sentença
                    for sent in text:
                        ent.append(entropy(sent))

            except:
                print("Não foi possível ler o arquivo", filename)
        
                
            # Contador de arquivos processados
            if n_files % 5 == 0:
                print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
    
    # Transformando a lista de entropias em uma np.array e excluir os valores "nan"
    ent = np.array(ent)
    ent = ent[~np.isnan(ent)]
    
    # Fim do contador de tempo
    print ("--- %s seconds ---" % (time.time() - start_time))
    
    return (math.exp(np.mean(ent)))

In [13]:
perplexity("Originais/teste")

C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- 5 arquivos processados --- --- 64.25417399406433 seconds ---
--- 10 arquivos processados --- --- 135.97785329818726 seconds ---
--- 181.1237940788269 seconds ---


16.54959512636126